In [1]:
import torch
import torch.nn as nn
from torch.autograd import Variable

from torchvision import transforms
from torchvision import datasets
print(torch.cuda.is_available())
torch.backends.cudnn.benchmark = True
is_cuda = True

True


In [2]:
train_dataset = datasets.MNIST(root='./data',
                           train=True,
                           transform=transforms.ToTensor(),
                           download=True)

test_dataset = datasets.MNIST(root = './data',
                              train = False,
                              transform=transforms.ToTensor(),
                              download = True)

In [10]:
batch_size = 100
n_iters = 3000
num_epochs = int(n_iters / (len(train_dataset) / batch_size))

In [4]:
train_loader = torch.utils.data.DataLoader(train_dataset,batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset,batch_size, shuffle=False)

In [6]:
model = nn.Sequential(
    nn.Linear(28*28, 100),
    nn.ELU(),
    nn.Linear(100, 10)
)
if is_cuda == True:
    model.cuda()

In [27]:
# class ANN(nn.Module):
#     def __init__(self, input_dim, hidden_size, num_classes):
#         super(ANN, self).__init__()
#         # Linear Function
#         self.fc1 = nn.Linear(input_dim, hidden_dim)
#         # Non-Linearity
#         self.elu = nn.ELU()
#         # Linear Function(Readout)
#         self.fc2 = nn.Linear(hidden_dim, output_dim)
        
#     def forward(self, x):
#         out = self.fc1(x)
#         out = self.elu(out)
#         out = self.fc2(out)
#         return out

In [28]:
# input_dim = 28*28
# hidden_dim = 100
# output_dim = 10

# model = ANN(input_dim, hidden_dim, output_dim)
# if is_cuda == True:
#     model.cuda()

In [7]:
criterion = nn.CrossEntropyLoss()

In [8]:
learning_rate = 0.1

optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate)

In [9]:
# No. of parameters layer
print(len(list(model.parameters())))

# No of parameters in 1st layer:
print(list(model.parameters())[0].size())

# No of parameters in 2nd layer:
print(list(model.parameters())[1].size())

# No of parameters in 3rd layer:
print(list(model.parameters())[2].size())

# No of parameters in 4th layer:
print(list(model.parameters())[3].size())

4
torch.Size([100, 784])
torch.Size([100])
torch.Size([10, 100])
torch.Size([10])


In [11]:
iter = 0
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = Variable(images.view(-1, 28*28))
        labels = Variable(labels)            
        if is_cuda == True:
            images = images.cuda()
            labels = labels.cuda()
        
        optimizer.zero_grad()
        
        outputs = model(images)
        
        loss = criterion(outputs, labels)
        
        loss.backward()
        
        optimizer.step()
        
        iter += 1
        
        if iter % 500 == 0 :
            correct = 0
            total = 0
            
            for images_t, labels_t in test_loader:
                if is_cuda == True:
                    images_t = Variable(images_t.view(-1, 28*28)).cuda()
                    labels_t = Variable(labels_t).cuda()
                else :
                    images_t = Variable(images_t.view(-1, 28*28))
                    labels_t = Variable(labels_t)
                    
                outputs_t = model(images_t)
                
                _, predicted_t = torch.max(outputs_t.detach(), 1)
                
                total += labels_t.size(0) # or, labels_t.size()[0]
                
                correct += (predicted_t == labels_t).sum()
                
            accuracy = 100 * int(correct) / total
            
            print("Iteration: {}, loss: {}, Accuracy: {}".format(iter, loss, accuracy))

Iteration: 500, loss: 0.2858935594558716, Accuracy: 90.8
Iteration: 1000, loss: 0.3245282769203186, Accuracy: 91.93
Iteration: 1500, loss: 0.2841361165046692, Accuracy: 92.61
Iteration: 2000, loss: 0.21976561844348907, Accuracy: 93.44
Iteration: 2500, loss: 0.1906813234090805, Accuracy: 94.0
Iteration: 3000, loss: 0.22590512037277222, Accuracy: 94.54


In [33]:
int(correct) / total

0.9648

In [34]:
predicted_t

tensor([8, 9, 0, 1, 2, 7, 4, 5, 6, 7, 8, 0, 1, 2, 3, 4, 7, 8, 9, 7, 8, 6, 4, 1,
        9, 8, 8, 4, 4, 7, 0, 1, 9, 2, 8, 7, 8, 2, 6, 0, 6, 5, 3, 3, 9, 9, 1, 4,
        0, 6, 1, 0, 0, 6, 2, 1, 1, 7, 7, 8, 4, 6, 0, 7, 0, 3, 6, 8, 7, 1, 5, 2,
        4, 9, 4, 2, 6, 4, 1, 7, 3, 6, 6, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 0, 1, 2,
        3, 4, 5, 6], device='cuda:0')

In [35]:
labels_t

tensor([8, 9, 0, 1, 2, 3, 4, 5, 6, 7, 8, 0, 1, 2, 3, 4, 7, 8, 9, 7, 8, 6, 4, 1,
        9, 3, 8, 4, 4, 7, 0, 1, 9, 2, 8, 7, 8, 2, 6, 0, 6, 5, 3, 3, 3, 9, 1, 4,
        0, 6, 1, 0, 0, 6, 2, 1, 1, 7, 7, 8, 4, 6, 0, 7, 0, 3, 6, 8, 7, 1, 5, 2,
        4, 9, 4, 3, 6, 4, 1, 7, 2, 6, 5, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 0, 1, 2,
        3, 4, 5, 6], device='cuda:0')

In [36]:
(predicted_t == labels_t).sum()

tensor(94, device='cuda:0')

In [37]:
labels

tensor([8, 8, 8, 7, 6, 7, 0, 4, 8, 7, 0, 1, 3, 7, 4, 0, 4, 9, 5, 0, 4, 8, 7, 5,
        8, 0, 1, 7, 9, 1, 6, 9, 3, 7, 9, 0, 5, 2, 9, 2, 4, 7, 0, 0, 1, 6, 8, 0,
        5, 7, 5, 3, 4, 2, 0, 5, 9, 2, 1, 4, 5, 8, 7, 3, 0, 7, 3, 2, 1, 0, 1, 1,
        4, 2, 9, 3, 3, 0, 7, 0, 5, 7, 9, 7, 8, 2, 9, 4, 9, 0, 3, 1, 7, 8, 2, 1,
        1, 9, 0, 6], device='cuda:0')